In [1]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
from ydata_profiling import ProfileReport


/home/e02964/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv("Bengaluru_House_Data.csv")

df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [3]:
p=ProfileReport(df)
p.to_file(output_file='output.html')

In [4]:
def extract(value):
    match = re.match(r'^\d+', value)
    return match.group(0)

In [5]:
def conv_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0]) + float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [6]:
df=df.dropna(subset=['location'])

df.loc[:, 'bath'] = df['bath'].fillna(2.0)
df['size']=df['size'].fillna("2 bhk")
df['balcony'] = df['balcony'].fillna(2.0) 
df['society'] = df['society'].fillna("Other")

df['size']=df['size'].apply(extract).astype(int)

counts=df['location'].value_counts()
other=counts[counts<=10].index
df['location']=df['location'].replace(other,'Other')

df['total_sqft'] = df['total_sqft'].apply(conv_to_num)
df=df.dropna(subset=['total_sqft'])

In [7]:
df['ppsft']=df['price']*100000/df['total_sqft']

In [8]:
q1=df['ppsft'].quantile(0.25)
q3=df['ppsft'].quantile(0.75)
iqr=q3-q1
ul=q3+1.5*iqr
ll=q1-1.5*iqr
df=df[(df['ppsft']>ll)&(df['ppsft']<ul)]

In [9]:
q1=df['total_sqft'].quantile(0.25)
q3=df['total_sqft'].quantile(0.75)
iqr=q3-q1
ul=q3+1.5*iqr
ll=q1-1.5*iqr
df=df[(df['total_sqft']>ll)&(df['total_sqft']<ul)]


In [10]:
df.to_csv("Cleaned_Data.csv",index=False)